In [1]:
import pandas as pd
import loguru
from data.dataset import Sentinel2Dataset
from data.loader import define_loaders
from utils.utils import load_config
import torch
import torch.nn as nn
from tqdm import tqdm

from model_zoo.models import define_model

/home/ubuntu/miniconda3/envs/ai_processor/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = load_config(config_path="cfg/config.yaml")
BASE_DIR = config["DATASET"]["base_dir"]
VERSION = config['DATASET']['version']
BATCH_SIZE = config['TRAINING']['batch_size']
VAL_BS = config["TRAINING"]['val_bs']
NUM_WORKERS = config['TRAINING']['num_workers']
RESIZE = config['TRAINING']['resize']
LEARNING_RATE = config['TRAINING']['learning_rate']
train_path = f"{BASE_DIR}/{VERSION}/train_path.csv"
val_path = f"{BASE_DIR}/{VERSION}/val_path.csv"
test_path = f"{BASE_DIR}/{VERSION}/test_path.csv"

In [3]:
df_train = pd.read_csv(train_path)
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

In [4]:
train_dataset = Sentinel2Dataset(df_path=df_train, train=True, augmentation=False)
val_dataset = Sentinel2Dataset(df_path=df_val, train=False, augmentation=False)
train_loader, val_loader = define_loaders(
        train_dataset,
        val_dataset,
        batch_size=VAL_BS,
        val_bs=VAL_BS,
        num_workers=NUM_WORKERS,
    )

TypeError: Sentinel2Dataset.__init__() missing 1 required positional argument: 'img_size'

In [ ]:
import torch.optim as optim

model = define_model(name="Unet", encoder_name="resnet34",
                     in_channel=3, out_channels=3, activation=None)

from utils.torch import count_parameters
nb_parameters = count_parameters(model=model)
print("Number of parameters {}".format(nb_parameters))
cuda = torch.device('cuda')

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=float(LEARNING_RATE))

In [ ]:
best_epoch = 0
best_loss = 0.0
step = 0
metrics_dict = {}
NUM_EPOCHS = 20

In [ ]:
model.train()
for epoch in range(NUM_EPOCHS):

     with tqdm(total=(len(train_dataset) - len(train_dataset) % BATCH_SIZE), colour='#3eedc4') as t:
            t.set_description('epoch: {}/{}'.format(epoch, NUM_EPOCHS - 1))

     model.train()
     optimizer.zero_grad()

     for x_data, y_data in train_loader:
          # Print tensor information
          x_data = x_data.cuda()
          # Forward pass
          y = model(x_data)


